<a href="https://colab.research.google.com/github/salhahCS/My-codes/blob/main/4DecYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


first, i should Mount Google Drive
Access dataset on drive
dataste 3 groups: train (7 videos), validation (video 11,12 from training), teating (9 vidoes)
i cleaned the dataset to contain only 2 classes: Non-Human Object and Moving In Opposite. rewrite again full guide.
we detect the abnormality in pedstraint in HAJJ.
the dataset originally splited into train and test, 9 videos for each group, i added validation to fune-tune my YOLO model.
/content/drive/MyDrive/ProjectDataset
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Testing
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation


In [ ]:
!pip install pandas opencv-python-headless
#Install Required Libraries

In [ ]:
#Convert CSV Labels to YOLO Format

import pandas as pd
import cv2
import os

def convert_to_yolo_format(video_file, csv_file, output_dir):
    # Read the label CSV file
    labels = pd.read_csv(csv_file)

    # Open video file to extract frame size (width, height)
    cap = cv2.VideoCapture(video_file)
    ret, frame = cap.read()
    height, width, _ = frame.shape  # Get frame size
    cap.release()

    # Process each frame in the label file
    for _, row in labels.iterrows():
        # Extract the frame number and corresponding bounding box coordinates
        frame_number = row['Frame_No']
        x_center = (row['X'] + row['Width'] / 2) / width
        y_center = (row['Y'] + row['Height'] / 2) / height
        w = row['Width'] / width
        h = row['Height'] / height

        # Map the class to its respective ID (0 for 'Moving Non-Human Object', 1 for 'Moving In Opposite')
        if row['Classes'] == 'Moving Non-Human Object':
            class_id = 0
        elif row['Classes'] == 'Moving In Opposite':
            class_id = 1
        else:
            continue  # Skip other classes

        # Define the YOLO annotation file name (same as frame number)
        yolo_annotation_path = os.path.join(output_dir, f"{frame_number}.txt")

        # Save the YOLO formatted data to the .txt file
        with open(yolo_annotation_path, 'a') as f:
            f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")


Convert CSV Labels to YOLO Format:
You need to convert your label files from the CSV format to YOLO format. YOLO requires labels in the following format for each frame:

<class_id> <x_center> <y_center> <width> <height>
Where:

class_id is 0 for "Moving Non-Human Object" and 1 for "Moving In Opposite".
x_center, y_center, width, height are normalized values (relative to the image dimensions).
Create the following function to process the video and labels, converting them to YOLO format

this code is wrong, works only in one video and does not iterate to all vidos.

In [ ]:
def process_all_videos(videos_dir, labels_dir, output_dir):
    # Loop through all videos in the directory and their corresponding label files
    for video_file in os.listdir(videos_dir):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(videos_dir, video_file)
            label_file = video_file.replace('.mp4', '.csv')
            label_path = os.path.join(labels_dir, label_file)

            # Convert CSV labels to YOLO format
            convert_to_yolo_format(video_path, label_path, output_dir)

# Process training, validation, and testing datasets
process_all_videos('/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Videos/',
                   '/content/drive/MyDrive/ProjectDataset/Training/Labels/',
                   '/content/drive/MyDrive/ProjectDataset/Train')

process_all_videos('/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/Videos/',
                   '/content/drive/MyDrive/ProjectDataset/Validation/Labels/',
                   '/content/drive/MyDrive/ProjectDataset/Validate')

process_all_videos('/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Testing/Videos/',
                   '/content/drive/MyDrive/ProjectDataset/Testing/Labels/',
                   '/content/drive/MyDrive/ProjectDataset/Test')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ProjectDataset/Training/Labels/2.csv'

wrong.
gonna enhance evverything down.

In [ ]:
#creating the base_dir i Drive

import os

# Set the path for base_dir
base_dir = '/content/drive/MyDrive/ProjectDataset'
# Create the base_dir if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    print(f"Created directory: {base_dir}")
else:
    print(f"Directory already exists: {base_dir}")


Directory already exists: /content/drive/MyDrive/ProjectDataset


The base_dir is used as the root from which all other relative paths are derived. For example, if your dataset is divided into training, validation, and testing data, each of these would have its own subdirectory within the base_dir.
The base_dir in your script represents the base directory path where your dataset is stored. It's a crucial part of the script because it specifies the starting point for all directory path constructions within your functions.

In [ ]:
import os

def process_all_videos_and_labels(base_dir, dataset_type):
    video_dir = os.path.join(base_dir, 'HAJJv2.Dataset', dataset_type, 'Videos')
    label_dir = os.path.join(base_dir, 'HAJJv2.Dataset', dataset_type, 'Labels')
    output_dir = os.path.join(base_dir, 'HAJJv2.Dataset', dataset_type, 'YOLO_Labels')
    os.makedirs(output_dir, exist_ok=True)  # Make sure the output directory exists

    # Loop through all video files in the video directory
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        csv_file = video_file.replace('.mp4', '.csv')  # Assumes label file matches video file name
        csv_path = os.path.join(label_dir, csv_file)

        if os.path.exists(csv_path):  # Check if the corresponding CSV file exists
            convert_to_yolo_format(video_path, csv_path, output_dir)
        else:
            print(f"Label file not found for {video_file}")

# Example usage:
base_directory = '/content/drive/MyDrive/ProjectDataset'
process_all_videos_and_labels(base_directory, 'Training')
process_all_videos_and_labels(base_directory, 'Validation')
process_all_videos_and_labels(base_directory, 'Testing')


Label file not found for 2.mp4
Label file not found for 10.mp4
Label file not found for 5.mp4
Label file not found for 8.mp4
Label file not found for 7.mp4
Label file not found for 9.mp4
Label file not found for 3.mp4
Label file not found for 12.mp4
Label file not found for 11.mp4
Label file not found for 11.mp4
Label file not found for 10.mp4
Label file not found for 12.mp4
Label file not found for 5.mp4
Label file not found for 8.mp4
Label file not found for 2.mp4
Label file not found for 9.mp4
Label file not found for 7.mp4
Label file not found for 3.mp4


In [ ]:
import os
import re

def process_all_videos_and_labels(base_dir, dataset_type):
    video_dir = os.path.join(base_dir, 'HAJJv2.Dataset', dataset_type, 'Videos')
    label_dir = os.path.join(base_dir, 'HAJJv2.Dataset', dataset_type, 'Labels')
    output_dir = os.path.join(base_dir, 'HAJJv2.Dataset', dataset_type, 'YOLO_Labels')
    os.makedirs(output_dir, exist_ok=True)

    # Loop through all video files in the video directory
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        # Extract the number from the video filename and construct the CSV filename
        video_number = re.search(r'\d+', video_file).group()
        csv_file = f"Dataset_V{video_number}_Train.csv"
        csv_path = os.path.join(label_dir, csv_file)

        print(f"Looking for label file: {csv_path}")  # Debug: print the path it's checking for CSV

        if os.path.exists(csv_path):
            convert_to_yolo_format(video_path, csv_path, output_dir)
        else:
            print(f"Label file not found for {video_file}")

# Example usage:
base_directory = '/content/drive/MyDrive/ProjectDataset'
process_all_videos_and_labels(base_directory, 'Training')
process_all_videos_and_labels(base_directory, 'Validation')
process_all_videos_and_labels(base_directory, 'Testing')


Looking for label file: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/Dataset_V2_Train.csv
Label file not found for 2.mp4
Looking for label file: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/Dataset_V10_Train.csv
Label file not found for 10.mp4
Looking for label file: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/Dataset_V5_Train.csv
Label file not found for 5.mp4
Looking for label file: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/Dataset_V8_Train.csv
Label file not found for 8.mp4
Looking for label file: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/Dataset_V7_Train.csv
Label file not found for 7.mp4
Looking for label file: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/Dataset_V9_Train.csv
Label file not found for 9.mp4
Looking for label file: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/Dataset_V3_Train.csv
Label file not

In [ ]:
import os

def process_videos_and_labels(video_dir, label_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Loop through all video files in the video directory
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        csv_file = video_file.replace('.mp4', '.csv')  # Assumes label file matches video file name
        csv_path = os.path.join(label_dir, csv_file)

        print(f"Looking for label file: {csv_path}")  # Debug: print the path it's checking for CSV

        if os.path.exists(csv_path):
            convert_to_yolo_format(video_path, csv_path, output_dir)
        else:
            print(f"Label file not found for {video_file}")

# Paths for each dataset
base_directory = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Videos = os.path.join(base_directory, 'Training', 'Videos')
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels = os.path.join(base_directory, 'Training', 'Labels')
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/YOLO_Labels = os.path.join(base_directory, 'Training', 'YOLO_Labels')

/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Testing/Videos = os.path.join(base_directory, 'Testing', 'Videos')
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Testing/Labels = os.path.join(base_directory, 'Testing', 'Labels')
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Testing/YOLO_Labels = os.path.join(base_directory, 'Testing', 'YOLO_Labels')

/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/Videos = os.path.join(base_directory, 'Validation', 'Videos')
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/Labels = os.path.join(base_directory, 'Validation', 'Labels')
/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/YOLO_Labels = os.path.join(base_directory, 'Validation', 'YOLO_Labels')

# Process each dataset
process_videos_and_labels('/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Videos', '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels', '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/YOLO_Labels')
process_videos_and_labels('/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Testing/Videos', '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Testing/Labels', '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Testing/YOLO_Labels')
process_videos_and_labels('/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/Videos', '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/Labels', '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/YOLO_Labels')


SyntaxError: invalid syntax (<ipython-input-9-5c372c69ef07>, line 22)

In [ ]:
import os

def convert_to_yolo_format(video_path, csv_path, output_dir):
    """
    Convert labels from CSV format to YOLO format.
    Assumes label file contains columns for bounding box coordinates and class names.
    """
    # Placeholder for the conversion process
    pass

def process_videos_and_labels(video_dir, label_dir, output_dir):
    """
    Process video files and corresponding labels, converting them to YOLO format.

    Args:
        video_dir (str): Directory containing video files.
        label_dir (str): Directory containing corresponding CSV label files.
        output_dir (str): Directory where YOLO formatted labels will be saved.
    """
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        csv_file = video_file.replace('.mp4', '.csv')
        csv_path = os.path.join(label_dir, csv_file)

        if os.path.exists(csv_path):
            try:
                convert_to_yolo_format(video_path, csv_path, output_dir)
            except Exception as e:
                print(f"Error processing {csv_file}: {e}")
        else:
            print(f"Label file not found for {video_file}")

def setup_dataset_processing(base_directory, dataset_type):
    """
    Setup paths and process a specific dataset type.

    Args:
        base_directory (str): Base directory where datasets are stored.
        dataset_type (str): Type of the dataset ('Training', 'Testing', 'Validation').
    """
    video_dir = os.path.join(base_directory, dataset_type, 'Videos')
    label_dir = os.path.join(base_directory, dataset_type, 'Labels')
    output_dir = os.path.join(base_directory, dataset_type, 'YOLO_Labels')

    process_videos_and_labels(video_dir, label_dir, output_dir)

# Base directory where the datasets are located
base_directory = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'

# Setup and process each dataset
dataset_types = ['Training', 'Testing', 'Validation']
for dataset_type in dataset_types:
    setup_dataset_processing(base_directory, dataset_type)


Label file not found for 2.mp4
Label file not found for 10.mp4
Label file not found for 5.mp4
Label file not found for 8.mp4
Label file not found for 7.mp4
Label file not found for 9.mp4
Label file not found for 3.mp4
Label file not found for 11.mp4
Label file not found for 10.mp4
Label file not found for 12.mp4
Label file not found for 5.mp4
Label file not found for 8.mp4
Label file not found for 2.mp4
Label file not found for 9.mp4
Label file not found for 7.mp4
Label file not found for 3.mp4
Label file not found for 12.mp4
Label file not found for 11.mp4


In [ ]:
import os
import pandas as pd
import cv2

def convert_to_yolo_format(csv_path, video_path, output_dir):
    """
    Convert labels from CSV format to YOLO format.

    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Read the CSV file
    labels = pd.read_csv(csv_path)

    # Open video to get dimensions
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print("Could not read video")
        cap.release()
        return
    height, width, _ = frame.shape
    cap.release()

    # Filter for specific classes
    labels = labels[labels['Classes'].isin(['Moving In Opposite', 'Moving Non-Human Object'])]

    # Class mapping to YOLO index
    class_mapping = {'Moving In Opposite': 0, 'Moving Non-Human Object': 1}

    # Process each label
    for index, row in labels.iterrows():
        # Calculate YOLO format parameters
        x_center = (row['X'] + (row['Width'] / 2)) / width
        y_center = (row['Y'] + (row['Height'] / 2)) / height
        box_width = row['Width'] / width
        box_height = row['Height'] / height
        class_id = class_mapping[row['Classes']]

        # Write to file
        label_filename = f"{int(row['Frame_No'])}.txt"
        with open(os.path.join(output_dir, label_filename), 'a') as file:
            file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

def process_dataset(base_dir, dataset_type):
    """
    Process a specific dataset to convert CSV labels to YOLO format.
    """
    video_dir = os.path.join(base_dir, dataset_type, '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Videos')
    label_dir = os.path.join(base_dir, dataset_type, '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels')
    output_dir = os.path.join(base_dir, dataset_type, '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/YOLO_Labels')

    # Process each video
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        csv_file = video_file.replace('.mp4', '.csv')
        csv_path = os.path.join(label_dir, csv_file)

        if os.path.exists(csv_path):
            convert_to_yolo_format(csv_path, video_path, output_dir)
        else:
            print(f"CSV file not found for video: {video_file}")

# Usage example
base_directory = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'
dataset_type = 'Training'  # Change to 'Validation' or 'Testing' as needed
process_dataset(base_directory, dataset_type)


CSV file not found for video: 2.mp4
CSV file not found for video: 10.mp4
CSV file not found for video: 5.mp4
CSV file not found for video: 8.mp4
CSV file not found for video: 7.mp4
CSV file not found for video: 9.mp4
CSV file not found for video: 3.mp4


In [ ]:
import os
import pandas as pd
import cv2

def convert_to_yolo_format(csv_path, video_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Try to read the CSV file
    try:
        labels = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Could not read CSV file at {csv_path}: {e}")
        return

    # Try to open the video file to get dimensions
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print(f"Could not read video file at {video_path}")
        cap.release()
        return
    height, width, _ = frame.shape
    cap.release()

    # Filter for specific classes and map classes to YOLO indices
    labels = labels[labels['Classes'].isin(['Moving In Opposite', 'Moving Non-Human Object'])]
    class_mapping = {'Moving In Opposite': 0, 'Moving Non-Human Object': 1}

    # Process each label and calculate YOLO format parameters
    for index, row in labels.iterrows():
        x_center = (row['X'] + row['Width'] / 2) / width
        y_center = (row['Y'] + row['Height'] / 2) / height
        box_width = row['Width'] / width
        box_height = row['Height'] / height
        class_id = class_mapping[row['Classes']]

        label_filename = f"{int(row['Frame_No'])}.txt"
        with open(os.path.join(output_dir, label_filename), 'a') as file:
            file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

def process_dataset(base_dir, dataset_type):
    video_dir = os.path.join(base_dir, dataset_type, 'Videos')
    label_dir = os.path.join(base_dir, dataset_type, 'Labels')
    output_dir = os.path.join(base_dir, dataset_type, 'YOLO_Labels')

    # Print paths to check correctness
    print(f"Video Directory: {video_dir}")
    print(f"Label Directory: {label_dir}")
    print(f"Output Directory: {output_dir}")

    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        csv_file = video_file.replace('.mp4', '.csv')
        csv_path = os.path.join(label_dir, csv_file)

        # Print expected CSV path for debugging
        print(f"Expected CSV path: {csv_path}")

        if os.path.exists(csv_path):
            convert_to_yolo_format(csv_path, video_path, output_dir)
        else:
            print(f"CSV file not found for video: {video_file}")

# Usage example
base_directory = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'
dataset_type = 'Training'  # Modify as necessary
process_dataset(base_directory, dataset_type)


Video Directory: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Videos
Label Directory: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels
Output Directory: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/YOLO_Labels
Expected CSV path: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/2.csv
CSV file not found for video: 2.mp4
Expected CSV path: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/10.csv
CSV file not found for video: 10.mp4
Expected CSV path: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/5.csv
CSV file not found for video: 5.mp4
Expected CSV path: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/8.csv
CSV file not found for video: 8.mp4
Expected CSV path: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/7.csv
CSV file not found for video: 7.mp4
Expected CSV path: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Lab

In [ ]:
import os
import pandas as pd
import cv2

def convert_to_yolo_format(csv_path, video_path, output_dir):
    """
    Convert labels from CSV format to YOLO format using video dimensions to normalize bounding box coordinates.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Read the CSV file
    labels = pd.read_csv(csv_path)

    # Open video to get dimensions
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print("Could not read video file: ", video_path)
        cap.release()
        return
    height, width, _ = frame.shape
    cap.release()

    # Filter for specific classes
    labels = labels[labels['Classes'].isin(['Moving In Opposite', 'Moving Non-Human Object'])]

    # Class mapping to YOLO index
    class_mapping = {'Moving In Opposite': 0, 'Moving Non-Human Object': 1}

    # Process each label
    for index, row in labels.iterrows():
        # Calculate YOLO format parameters
        x_center = (row['X'] + row['Width'] / 2) / width
        y_center = (row['Y'] + row['Height'] / 2) / height
        box_width = row['Width'] / width
        box_height = row['Height'] / height
        class_id = class_mapping[row['Classes']]

        # Write to file
        label_filename = f"{int(row['Frame_No'])}.txt"
        with open(os.path.join(output_dir, label_filename), 'a') as file:
            file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

def process_dataset(base_dir, dataset_type):
    """
    Process a specific dataset to convert CSV labels to YOLO format, matching videos to their labels based on a naming convention.
    """
    video_dir = os.path.join(base_dir, dataset_type, 'Videos')
    label_dir = os.path.join(base_dir, dataset_type, 'Labels')
    output_dir = os.path.join(base_dir, dataset_type, 'YOLO_Labels')

     # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process each video
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        csv_file = f"HajjDataset_V{video_file.split('.')[0]}_Train.csv"  # Construct label file name
        csv_path = os.path.join(label_dir, csv_file)

        if os.path.exists(csv_path):
            convert_to_yolo_format(csv_path, video_path, output_dir)
        else:
            print(f"CSV file not found for video: {video_file}")

# Usage example
base_directory = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'
process_dataset(base_directory, 'Training')


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

def check_directories(base_dir, dataset_type):
    video_dir = os.path.join(base_dir, dataset_type, 'Videos')
    label_dir = os.path.join(base_dir, dataset_type, 'Labels')
    output_dir = os.path.join(base_dir, dataset_type, 'YOLO_Labels')

    print(f"Video directory: {video_dir}")
    print(f"Label directory: {label_dir}")
    print(f"Output directory: {output_dir}")

    # Check if directories exist
    print("Checking directories...")
    for path in [video_dir, label_dir, output_dir]:
        if os.path.exists(path):
            print(f"Directory exists: {path}")
            print("Contents:", os.listdir(path))
        else:
            print(f"Directory does not exist: {path}")

base_directory = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'
dataset_type = 'Training'
check_directories(base_directory, dataset_type)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Video directory: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Videos
Label directory: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels
Output directory: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/YOLO_Labels
Checking directories...
Directory exists: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Videos
Contents: ['2.mp4', '10.mp4', '5.mp4', '8.mp4', '7.mp4', '9.mp4', '3.mp4']
Directory exists: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels
Contents: ['HajjDataset_V9_Train.csv', 'HajjDataset_V2_Train.csv', 'HajjDataset_V10_Train.csv', 'HajjDataset_V5_Train.csv', 'HajjDataset_V8_Train.csv', 'HajjDataset_V3_Train.csv', 'HajjDataset_V7_Train.csv']
Directory exists: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/YOLO_Labels
Contents: []


In [2]:
import cv2

def get_video_dimensions(video_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    return width, height

# Example usage
video_path = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/Videos/11.mp4'
width, height = get_video_dimensions(video_path)
print(f"Video dimensions: {width}x{height}")


Video dimensions: 1920x1088


First, to know the dimintioms of our vidoes, we run this code to be sure to processes our dataset.
Before processing your videos, it’s important to know the dimensions (width and height) of the frames. You can find this information using OpenCV:
This function opens a video file and retrieves its frame dimensions, which are crucial for accurately normalizing the bounding box coordinates in the YOLO label files.

In [3]:
!pip install opencv-python-headless  # For image processing
!pip install pandas  # For handling CSV data


In [5]:
import cv2
import os
import pandas as pd

# Define the path to the video and the CSV file
video_path = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/Videos/11.mp4'
csv_file_path = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/Labels/HajjDataset_V11_Train.csv'
output_frames_path = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Validation/YOLO_Labels/11'
frame_dimensions = (1920, 1088)  # Adjust based on your actual frame size 1920x1088

# Create the output directory if it does not exist
if not os.path.exists(output_frames_path):
    os.makedirs(output_frames_path)

# Function to extract frames from a video one frame per second
def extract_frames(video_path, out_path, fps=1):
    cap = cv2.VideoCapture(video_path)
    video_fps = int(cap.get(cv2.CAP_PROP_FPS))
    count = 0

    while True:
        success, image = cap.read()
        if not success:
            break
        if count % (video_fps // fps) == 0:
            frame_id = count // (video_fps // fps)
            cv2.imwrite(f"{out_path}/frame_{frame_id:05d}.jpg", image)
        count += 1
    cap.release()

# Function to convert CSV labels to YOLO format
def convert_labels_to_yolo(csv_path, out_path, dimensions):
    df = pd.read_csv(csv_path)
    interested_classes = {'moving_in_opposite': 0, 'moving_non-human_object': 1}
    df = df[df['Classes'].isin(interested_classes.keys())]

    for idx, row in df.iterrows():
        frame_index = row['Frame_No']
        class_id = interested_classes[row['Classes']]
        x_center = (row['X'] + row['Width'] / 2) / dimensions[0]
        y_center = (row['Y'] + row['Height'] / 2) / dimensions[1]
        width = row['Width'] / dimensions[0]
        height = row['Height'] / dimensions[1]
        label_format = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

        label_path = f"{out_path}/frame_{frame_index:05d}.txt"
        with open(label_path, 'a') as file:
            file.write(label_format + "\n")

# Run the functions
extract_frames(video_path, output_frames_path)
convert_labels_to_yolo(csv_file_path, output_frames_path, frame_dimensions)


In [ ]:
it makes fram extraction for the specified video.
bad

new start on the structure i got from their website for training

In [2]:
!pip install opencv-python-headless
!pip install pandas


In [5]:
import cv2
import os
import pandas as pd

# Base directory for dataset
base_dir = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'

# Directory structure
dirs = {
    'Training': 'train',
    'Validation': 'val',
    'Testing': 'test'
}

# Process videos and labels for each dataset part
for dataset_part, yolo_part in dirs.items():
    video_dir = f'{base_dir}/{dataset_part}/Videos'
    label_dir = f'{base_dir}/{dataset_part}/Labels'
    output_image_dir = f'{base_dir}/{yolo_part}/images'
    output_label_dir = f'{base_dir}/{yolo_part}/labels'

    # Create output directories if they don't exist
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    # List all videos
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        video_base = os.path.splitext(video_file)[0]

        # Extract frames
        cap = cv2.VideoCapture(video_path)
        video_fps = int(cap.get(cv2.CAP_PROP_FPS))
        count = 0

        while True:
            success, image = cap.read()
            if not success:
                break
            frame_filename = f'{video_base}_frame{count:03d}.jpg'
            cv2.imwrite(os.path.join(output_image_dir, frame_filename), image)
            count += 1

        cap.release()

        # Convert labels
        label_path = os.path.join(label_dir, video_base + '.csv')
        df = pd.read_csv(label_path)
        interested_classes = {'moving_in_opposite': 0, 'moving_non_human_object': 1}
        df = df[df['Classes'].isin(interested_classes.keys())]

        frame_dimensions = (1920, 1080)  # Update based on actual frame dimensions

        for _, row in df.iterrows():
            frame_index = row['Frame_No']
            class_id = interested_classes[row['Classes']]
            x_center = (row['X'] + row['Width'] / 2) / frame_dimensions[0]
            y_center = (row['Y'] + row['Height'] / 2) / frame_dimensions[1]
            width = row['Width'] / frame_dimensions[0]
            height = row['Height'] / frame_dimensions[1]
            label_format = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

            label_filename = f'{video_base}_frame{frame_index:03d}.txt'
            label_filepath = os.path.join(output_label_dir, label_filename)
            with open(label_filepath, 'a') as file:
                file.write(label_format + "\n")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/2.csv'

الكود ممتاز و بسرعه تطلع الفريمات و طلع لي من التدريب 50 فريم لكن الليبل غلط يحتاج بس تعديل لكن بجرب كود ثاني


In [6]:
import cv2
import os
import pandas as pd

# Function to retrieve video frame dimensions
def get_video_dimensions(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video {video_path}")
        return None
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    return width, height

# Base directory for dataset
base_dir = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'

# Directory structure mapping from your folder names to YOLO's structure
dirs = {
    'Training': 'train',
    'Validation': 'val',
    'Testing': 'test'
}

# Process videos and labels for each dataset part
for dataset_part, yolo_part in dirs.items():
    video_dir = f'{base_dir}/{dataset_part}/Videos'
    label_dir = f'{base_dir}/{dataset_part}/Labels'
    output_image_dir = f'{base_dir}/{yolo_part}/images'
    output_label_dir = f'{base_dir}/{yolo_part}/labels'

    # Ensure output directories exist
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    # List and process all videos
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        video_base = os.path.splitext(video_file)[0]

        # Extract video frames
        cap = cv2.VideoCapture(video_path)
        video_fps = int(cap.get(cv2.CAP_PROP_FPS))
        count = 0

        while True:
            success, image = cap.read()
            if not success:
                break
            frame_filename = f'{video_base}_frame{count:03d}.jpg'
            cv2.imwrite(os.path.join(output_image_dir, frame_filename), image)
            count += 1
        cap.release()

        # Convert labels
        frame_dimensions = get_video_dimensions(video_path)
        if frame_dimensions:
            label_file_pattern = video_base.replace('video', 'HajjDataset_V') + '_Train.csv'
            label_path = os.path.join(label_dir, label_file_pattern)
            if os.path.exists(label_path):
                df = pd.read_csv(label_path)
                interested_classes = {'moving_in_opposite': 0, 'moving_non_human_object': 1}
                df = df[df['Classes'].isin(interested_classes.keys())]

                for _, row in df.iterrows():
                    frame_index = row['Frame_No']
                    class_id = interested_classes[row['Classes']]
                    x_center = (row['X'] + row['Width'] / 2) / frame_dimensions[0]
                    y_center = (row['Y'] + row['Height'] / 2) / frame_dimensions[1]
                    width = row['Width'] / frame_dimensions[0]
                    height = row['Height'] / frame_dimensions[1]
                    label_format = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

                    label_filename = f'{video_base}_frame{frame_index:03d}.txt'
                    label_filepath = os.path.join(output_label_dir, label_filename)
                    with open(label_filepath, 'a') as file:
                        file.write(label_format + "\n")
            else:
                print(f"Label file does not exist: {label_path}")
        else:
            print(f"Failed to retrieve dimensions for video {video_path}")


Label file does not exist: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/2_Train.csv
Label file does not exist: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/10_Train.csv
Label file does not exist: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/5_Train.csv
Label file does not exist: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/8_Train.csv
Label file does not exist: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/7_Train.csv
Label file does not exist: /content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset/Training/Labels/9_Train.csv


KeyboardInterrupt: 

still theres an issue with the labeling only, great code though. still extracting 50 frames for traning

In [7]:
import cv2
import os
import pandas as pd

def get_frame_dimensions(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    return width, height

# Base directory for dataset
base_dir = '/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset'  #/content/drive/MyDrive/ProjectDataset/HAJJv2.Dataset

# Define suffixes for label files based on dataset parts
label_suffix = {
    'Training': 'Train',
    'Testing': 'Test',
    'Validation': 'Validat'
}

# Process videos and labels for each dataset part
for dataset_part, suffix in label_suffix.items():
    video_dir = os.path.join(base_dir, dataset_part, 'Videos')
    label_dir = os.path.join(base_dir, dataset_part, 'Labels')
    output_image_dir = os.path.join(base_dir, dataset_part.lower(), 'images')
    output_label_dir = os.path.join(base_dir, dataset_part.lower(), 'labels')

    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    # List and process all videos
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        video_base = os.path.splitext(video_file)[0]
        video_number = ''.join(filter(str.isdigit, video_base))  # Extract number from filename

        # Extract frames
        cap = cv2.VideoCapture(video_path)
        video_fps = int(cap.get(cv2.CAP_PROP_FPS))
        count = 0

        while True:
            success, image = cap.read()
            if not success:
                break
            frame_filename = f'{video_base}_frame{count:03d}.jpg'
            cv2.imwrite(os.path.join(output_image_dir, frame_filename), image)
            count += 1

        cap.release()

        # Construct the label file name and path
        label_file_name = f'HajjDataset_V{video_number}_{suffix}.csv'
        label_path = os.path.join(label_dir, label_file_name)

        if os.path.exists(label_path):
            frame_dimensions = get_frame_dimensions(video_path)
            if frame_dimensions:
                df = pd.read_csv(label_path)
                interested_classes = {'moving_in_opposite': 0, 'moving_non_human_object': 1}
                df = df[df['Classes'].isin(interested_classes.keys())]

                for _, row in df.iterrows():
                    frame_index = row['Frame_No']
                    class_id = interested_classes[row['Classes']]
                    x_center = (row['X'] + row['Width'] / 2) / frame_dimensions[0]
                    y_center = (row['Y'] + row['Height'] / 2) / frame_dimensions[1]
                    width = row['Width'] / frame_dimensions[0]
                    height = row['Height'] / frame_dimensions[1]
                    label_format = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

                    label_filename = f'{video_base}_frame{frame_index:03d}.txt'
                    label_filepath = os.path.join(output_label_dir, label_filename)
                    with open(label_filepath, 'a') as file:
                        file.write(label_format + "\n")
            else:
                print(f"Failed to retrieve dimensions for video {video_path}")
        else:
            print(f"Label file does not exist: {label_path}")


the PERFECTION speaks

done   15m 13s
completed at 3:59 AM
all frames are extracted, all annotation are made


finaliy, do the yalm file:

path: ../datasets/HAJJv2  # dataset root dir
train: images/train  # train images
val: images/val    # val images
test: images/test  # test images

# Classes
names:
  0: moving_in_opposite
  1: moving_non_human_object
  
